**Inicialmente vamos a importar las librerías necesarias para que el código funcione, en el caso de necesitar librerías nuevas, serán agregadas a medida que sean requeridas**

In [1]:
!pip install kaggle
# 1. Importar las librerías necesarias
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import json
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer


In [2]:
os.environ['KAGGLE_CONFIG_DIR']='/content/'

In [3]:
# Cargar credenciales de Kaggle
data = {"username":"juanmanuellopezarias","key":"3a9f59f3e8a3191d8732a210262ac91a"}
with open('kaggle.json','w') as file:
  json.dump(data, file, indent=4)

**Descargamos los datos de Kaggle**

In [4]:
!kaggle competitions download -c udea-ai4eng-20242

 60% 12.0M/20.1M [00:00<00:00, 124MB/s]
100% 20.1M/20.1M [00:00<00:00, 166MB/s]


In [5]:
!unzip \*.zip && rm *.zip

Archive:  udea-ai4eng-20242.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


# **Fase de carga de datos e inspección inicial**

In [6]:
#Cargar el archivo CSV
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
df=train.copy()
df=df.drop(['ID'], axis=1)
df=df.drop(['PERIODO'], axis=1)

#Revisión inicial de los datos
print(df.head())
print(df.info())
print(df.describe())

#Manejo de datos faltantes
#Identificar columnas con valores nulos
print(df.isnull().sum())



          ESTU_PRGM_ACADEMICO ESTU_PRGM_DEPARTAMENTO  \
0                  ENFERMERIA                 BOGOTÁ   
1                     DERECHO              ATLANTICO   
2       MERCADEO Y PUBLICIDAD                 BOGOTÁ   
3  ADMINISTRACION DE EMPRESAS              SANTANDER   
4                  PSICOLOGIA              ANTIOQUIA   

             ESTU_VALORMATRICULAUNIVERSIDAD ESTU_HORASSEMANATRABAJA  \
0  Entre 5.5 millones y menos de 7 millones       Menos de 10 horas   
1  Entre 2.5 millones y menos de 4 millones                       0   
2  Entre 2.5 millones y menos de 4 millones         Más de 30 horas   
3  Entre 4 millones y menos de 5.5 millones                       0   
4  Entre 2.5 millones y menos de 4 millones     Entre 21 y 30 horas   

  FAMI_ESTRATOVIVIENDA FAMI_TIENEINTERNET                 FAMI_EDUCACIONPADRE  \
0            Estrato 3                 Si    Técnica o tecnológica incompleta   
1            Estrato 3                 No      Técnica o tecnológica compl

In [9]:
train_labels = train[['ID', 'RENDIMIENTO_GLOBAL']]

# Eliminar la columna label de train para unirlo con test
train = train.drop(columns=['RENDIMIENTO_GLOBAL'])

# Unir train y test
datos_combinados = pd.concat([train, test], ignore_index=True)

In [10]:
null_counts = datos_combinados.isnull().sum()
null_counts = null_counts[null_counts > 0]

null_counts

,0
ESTU_VALORMATRICULAUNIVERSIDAD,9010
ESTU_HORASSEMANATRABAJA,44236
FAMI_ESTRATOVIVIENDA,45932
FAMI_TIENEINTERNET,38168
FAMI_EDUCACIONPADRE,33171
FAMI_EDUCACIONMADRE,33887
ESTU_PAGOMATRICULAPROPIO,9305
Unnamed: 0,692500


In [11]:
for column in null_counts.index:
    mode_value = datos_combinados[column].mode()[0]
    datos_combinados[column].fillna(mode_value, inplace=True)

remaining_nulls = datos_combinados.isnull().sum().sum()
remaining_nulls

<ipython-input-11-aa448eb2939a>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  datos_combinados[column].fillna(mode_value, inplace=True)


0

# **Fase de imputación de datos y tratamiento especial de algunas variables categóricas binarias**

In [12]:
#Variables Categoricas
ccols = [i for i in datos_combinados.columns if not i in datos_combinados._get_numeric_data()]
print (ccols)
#Valor mas repetido
valor_comun = datos_combinados[ccols].mode().iloc[0]
valor_comun
#Reemplazar datos faltantes con los mas repetidos
for column in ccols:
  datos_combinados[column].fillna(valor_comun[column], inplace=True)

datos_combinados

['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE', 'ESTU_PAGOMATRICULAPROPIO']


<ipython-input-12-04b137fec62f>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  datos_combinados[column].fillna(valor_comun[column], inplace=True)


,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,Unnamed: 0
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,0.0
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,0.0
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,0.0
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,0.0
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
989281,496981,20195,ADMINISTRACION DE EMPRESAS,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 1,Si,Primaria incompleta,Primaria incompleta,Si,296781.0
989282,209415,20183,DERECHO,META,Entre 1 millón y menos de 2.5 millones,0,Estrato 4,Si,Educación profesional completa,Educación profesional completa,No,296782.0
989283,239074,20212,DERECHO,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Educación profesional completa,No,296783.0
989284,963852,20195,INGENIERIA AERONAUTICA,ANTIOQUIA,Entre 5.5 millones y menos de 7 millones,Entre 11 y 20 horas,Estrato 3,Si,Educación profesional completa,Educación profesional completa,No,296784.0


In [13]:
#eliminar la tercera columna del dataframe
datos_combinados = datos_combinados.drop(datos_combinados.columns[2], axis=1)
datos_combinados=datos_combinados.drop(datos_combinados.columns[-1],axis=1)
datos_combinados.head()

,ID,PERIODO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,904256,20212,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No
1,645256,20212,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No
2,308367,20203,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No
3,470353,20195,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No
4,989032,20212,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No


 Tratamiento especial de la columna **ESTU_PRGM_ACADEMICO** para reducir las columnas al hacer el One Hot encoding

In [ ]:
# def categorizar_programa(academico):
#     academico = academico.upper()  # Convertir a mayúsculas para evitar distinciones de minúsculas/mayúsculas

#     if academico.startswith('INGE'):
#         return 'INGENIERIA'
#     elif academico.startswith('LIC'):
#         return 'LICENCIATURA'
#     elif academico.startswith('ADM'):
#         return 'ADMINISTRACION'
#     elif academico.startswith('CONT'):
#         return 'CONTADURIA'
#     elif academico.startswith('COMU'):
#         return 'COMUNICACION'
#     elif 'DEPORT' in academico:
#         return 'DEPORTE'
#     elif 'PSIC' in academico:
#         return 'PSICOLOGIA'
#     elif 'TRABAJO SOCIAL' in academico:
#         return 'TRABAJO SOCIAL'
#     elif 'DISE' in academico:
#         return 'DISEÑO'
#     elif 'TERAP' in academico:
#         return 'TERAPIA'
#     elif 'GASTRON' in academico:
#         return 'GASTRONOMIA'
#     elif 'ZOOTECNIA' in academico:
#         return 'VETERINARIA Y ZOOTECNIA'
#     elif 'NUTRIC' in academico:
#         return 'NUTRICION'
#     elif 'MARKETING' in academico or 'PUBLICIDAD' in academico or 'MERCAD' in academico:
#         return 'MARKETING'
#     elif 'MATE' in academico or 'ESTA' in academico:
#         return 'MATEMATICAS Y ESTADISTICA'
#     elif 'ARTE' in academico:
#         return 'ARTES'
#     elif 'PERIOD' in academico:
#         return 'PERIODISMO'
#     elif 'LITER' in academico:
#         return 'LITERATURA'
#     elif 'MUS' in academico or 'MÚS' in academico:
#         return 'MUSICA'
#     elif 'ECONO' in academico or 'NEG' in academico or 'COME' in academico or 'FIN' in academico:
#         return 'ECONOMIA Y NEGOCIOS'
#     elif 'GEO' in academico:
#         return 'CIENCIAS DE LA TIERRA'
#     elif 'CINE' in academico or 'RADIO' in academico or 'TELEV' in academico or 'AUDIOV' in academico or 'FOTOG' in academico:
#         return 'CIENCIAS AUDIOVISUALES'
#     elif 'DERECHO' in academico:
#         return 'DERECHO'
#     elif 'POLIT' in academico or 'POLÍT' in academico:
#         return 'CIENCIAS POLITICAS'
#     elif 'QUIM' in academico or 'QUÍM' in academico:
#         return 'QUIMICA'
#     elif 'CONST' in academico:
#         return 'CONSTRUCCION'
#     elif 'TURIS' in academico:
#         return 'TURISMO'
#     elif 'BIBLIO' in academico:
#         return 'BIBLIOTECOLOGIA'
#     elif 'FILOS' in academico:
#         return 'FILOSOFIA'
#     elif 'FILOL' in academico:
#         return 'FILOLOGIA'
#     elif 'ODONTOLOG' in academico:
#         return 'ODONTOLOGIA'
#     elif 'BIOL' in academico:
#         return 'BIOLOGIA Y MICROBIOLOGIA'
#     elif academico.startswith('ECOL'):
#         return 'ECOLOGIA'
#     elif 'BACT' in academico:
#         return 'BACTERIOLOGIA'
#     elif 'CULT' in academico:
#         return 'CIENCIAS CULTURALES'
#     elif 'INSTRUMENTACION' in academico:
#         return 'INSTRUMENTACION QUIRURGICA'
#     elif 'MILIT' in academico or 'NAVALES' in academico or 'NAU' in academico:
#         return 'CIENCIAS MARITIMAS Y AERONAUTICAS'
#     else:
#         return academico  # Si no coincide con ninguna de las condiciones, se deja igual
# #categorizar academico
# datos_combinados['ESTU_PRGM_ACADEMICO'] = datos_combinados['ESTU_PRGM_ACADEMICO'].apply(categorizar_programa)
# # Usamos una lista de patrones que deseamos reemplazar y los eliminamos en una sola llamada
# patterns = ['PROFESIONAL EN ', 'PROFESIONAL DE ', 'PROFESIONAL ', 'PROGRAMA ']
# # Reemplazamos todos los patrones con un espacio vacío usando regex
# datos_combinados['ESTU_PRGM_ACADEMICO'] = datos_combinados['ESTU_PRGM_ACADEMICO'].replace(patterns, '', regex=True)
# # Aplicar la función a la columna
# datos_combinados['ESTU_PRGM_ACADEMICO'] = datos_combinados['ESTU_PRGM_ACADEMICO'].apply(categorizar_programa)

 Tratamiento de las columnas con valores **binarios** *(si no)*, se decide en la columna de **ESTU_HORASSEMANATRABAJA**, convertir a binaria (Trabajo o no )


In [14]:
datos_combinados["FAMI_TIENEINTERNET"] = datos_combinados["FAMI_TIENEINTERNET"].replace({'Si': 1, 'No': 0})
datos_combinados["ESTU_PAGOMATRICULAPROPIO"] = datos_combinados["ESTU_PAGOMATRICULAPROPIO"].replace({'Si': 1, 'No': 0})
datos_combinados['ESTU_HORASSEMANATRABAJA'] = datos_combinados['ESTU_HORASSEMANATRABAJA'].apply(lambda x: 1 if x != 0 else 0)
datos_combinados.head()

<ipython-input-14-a975cb33ae40>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  datos_combinados["FAMI_TIENEINTERNET"] = datos_combinados["FAMI_TIENEINTERNET"].replace({'Si': 1, 'No': 0})
<ipython-input-14-a975cb33ae40>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  datos_combinados["ESTU_PAGOMATRICULAPROPIO"] = datos_combinados["ESTU_PAGOMATRICULAPROPIO"].replace({'Si': 1, 'No': 0})


,ID,PERIODO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,904256,20212,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,1,Estrato 3,1,Técnica o tecnológica incompleta,Postgrado,0
1,645256,20212,ATLANTICO,Entre 2.5 millones y menos de 4 millones,1,Estrato 3,0,Técnica o tecnológica completa,Técnica o tecnológica incompleta,0
2,308367,20203,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,1,Estrato 3,1,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,0
3,470353,20195,SANTANDER,Entre 4 millones y menos de 5.5 millones,1,Estrato 4,1,No sabe,Secundaria (Bachillerato) completa,0
4,989032,20212,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,1,Estrato 3,1,Primaria completa,Primaria completa,0


In [ ]:
datos_combinados.shape

Identificamos las columnas categóricas eliminamos las que tienen un **tratamiento especial** ya realizado

In [15]:
# Identificar columnas categóricas
categorical_columns = datos_combinados.select_dtypes(include=['object']).columns
print(categorical_columns)

Index(['ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE'],
      dtype='object')


# **One-Hot Encoding y Normalización**

Aplicamos **One-Hot Encoding** a las columnas categóricas restantes, normalizamos características numéricas (si aun existen)

In [22]:
categorical_columns = datos_combinados.select_dtypes(include=['object']).columns

# Apply OneHotEncoder to categorical columns
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_data = encoder.fit_transform(datos_combinados[categorical_columns])
df_encoded = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_columns))

# Concatenate encoded features with numerical features and ID, PERIODO
df_preprocessed = pd.concat([datos_combinados[['ID', 'PERIODO', 'FAMI_TIENEINTERNET', 'ESTU_PAGOMATRICULAPROPIO']],
                             df_encoded], axis=1)

# Display all columns
pd.set_option('display.max_columns', None)

Unificamos los Dataframes creados y codificados, añadiendo la columna **ID y PERIODO** que borramos al inicio para evitar una normalización de estas características numéricas.

# **Visualización Final de los Datos**

Mostramos el **Dataframe Final**

In [23]:
df_preprocessed.head()

,ID,PERIODO,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,ESTU_PRGM_DEPARTAMENTO_ARAUCA,ESTU_PRGM_DEPARTAMENTO_ATLANTICO,ESTU_PRGM_DEPARTAMENTO_BOGOTÁ,ESTU_PRGM_DEPARTAMENTO_BOLIVAR,ESTU_PRGM_DEPARTAMENTO_BOYACA,ESTU_PRGM_DEPARTAMENTO_CALDAS,ESTU_PRGM_DEPARTAMENTO_CAQUETA,ESTU_PRGM_DEPARTAMENTO_CASANARE,ESTU_PRGM_DEPARTAMENTO_CAUCA,ESTU_PRGM_DEPARTAMENTO_CESAR,ESTU_PRGM_DEPARTAMENTO_CHOCO,ESTU_PRGM_DEPARTAMENTO_CORDOBA,ESTU_PRGM_DEPARTAMENTO_CUNDINAMARCA,ESTU_PRGM_DEPARTAMENTO_GUAVIARE,ESTU_PRGM_DEPARTAMENTO_HUILA,ESTU_PRGM_DEPARTAMENTO_LA GUAJIRA,ESTU_PRGM_DEPARTAMENTO_MAGDALENA,ESTU_PRGM_DEPARTAMENTO_META,ESTU_PRGM_DEPARTAMENTO_NARIÑO,ESTU_PRGM_DEPARTAMENTO_NORTE SANTANDER,ESTU_PRGM_DEPARTAMENTO_PUTUMAYO,ESTU_PRGM_DEPARTAMENTO_QUINDIO,ESTU_PRGM_DEPARTAMENTO_RISARALDA,ESTU_PRGM_DEPARTAMENTO_SAN ANDRES,ESTU_PRGM_DEPARTAMENTO_SANTANDER,ESTU_PRGM_DEPARTAMENTO_SUCRE,ESTU_PRGM_DEPARTAMENTO_TOLIMA,ESTU_PRGM_DEPARTAMENTO_VALLE,ESTU_PRGM_DEPARTAMENTO_VAUPES,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 2.5 millones y menos de 4 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 4 millones y menos de 5.5 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 5.5 millones y menos de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 500 mil y menos de 1 millón,ESTU_VALORMATRICULAUNIVERSIDAD_Menos de 500 mil,ESTU_VALORMATRICULAUNIVERSIDAD_Más de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_No pagó matrícula,FAMI_ESTRATOVIVIENDA_Estrato 2,FAMI_ESTRATOVIVIENDA_Estrato 3,FAMI_ESTRATOVIVIENDA_Estrato 4,FAMI_ESTRATOVIVIENDA_Estrato 5,FAMI_ESTRATOVIVIENDA_Estrato 6,FAMI_ESTRATOVIVIENDA_Sin Estrato,FAMI_EDUCACIONPADRE_Educación profesional incompleta,FAMI_EDUCACIONPADRE_Ninguno,FAMI_EDUCACIONPADRE_No Aplica,FAMI_EDUCACIONPADRE_No sabe,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Técnica o tecnológica completa,FAMI_EDUCACIONPADRE_Técnica o tecnológica incompleta,FAMI_EDUCACIONMADRE_Educación profesional incompleta,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta
0,904256,20212,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,645256,20212,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,308367,20203,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,470353,20195,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,989032,20212,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [24]:
train_listos = df_preprocessed[df_preprocessed['ID'].isin(train_labels['ID'])]
test_listos = df_preprocessed[~df_preprocessed['ID'].isin(train_labels['ID'])]


train_listos = train_listos.merge(train_labels, on='ID', how='left')

In [25]:
test_listos.head()


,ID,PERIODO,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,ESTU_PRGM_DEPARTAMENTO_ARAUCA,ESTU_PRGM_DEPARTAMENTO_ATLANTICO,ESTU_PRGM_DEPARTAMENTO_BOGOTÁ,ESTU_PRGM_DEPARTAMENTO_BOLIVAR,ESTU_PRGM_DEPARTAMENTO_BOYACA,ESTU_PRGM_DEPARTAMENTO_CALDAS,ESTU_PRGM_DEPARTAMENTO_CAQUETA,ESTU_PRGM_DEPARTAMENTO_CASANARE,ESTU_PRGM_DEPARTAMENTO_CAUCA,ESTU_PRGM_DEPARTAMENTO_CESAR,ESTU_PRGM_DEPARTAMENTO_CHOCO,ESTU_PRGM_DEPARTAMENTO_CORDOBA,ESTU_PRGM_DEPARTAMENTO_CUNDINAMARCA,ESTU_PRGM_DEPARTAMENTO_GUAVIARE,ESTU_PRGM_DEPARTAMENTO_HUILA,ESTU_PRGM_DEPARTAMENTO_LA GUAJIRA,ESTU_PRGM_DEPARTAMENTO_MAGDALENA,ESTU_PRGM_DEPARTAMENTO_META,ESTU_PRGM_DEPARTAMENTO_NARIÑO,ESTU_PRGM_DEPARTAMENTO_NORTE SANTANDER,ESTU_PRGM_DEPARTAMENTO_PUTUMAYO,ESTU_PRGM_DEPARTAMENTO_QUINDIO,ESTU_PRGM_DEPARTAMENTO_RISARALDA,ESTU_PRGM_DEPARTAMENTO_SAN ANDRES,ESTU_PRGM_DEPARTAMENTO_SANTANDER,ESTU_PRGM_DEPARTAMENTO_SUCRE,ESTU_PRGM_DEPARTAMENTO_TOLIMA,ESTU_PRGM_DEPARTAMENTO_VALLE,ESTU_PRGM_DEPARTAMENTO_VAUPES,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 2.5 millones y menos de 4 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 4 millones y menos de 5.5 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 5.5 millones y menos de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 500 mil y menos de 1 millón,ESTU_VALORMATRICULAUNIVERSIDAD_Menos de 500 mil,ESTU_VALORMATRICULAUNIVERSIDAD_Más de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_No pagó matrícula,FAMI_ESTRATOVIVIENDA_Estrato 2,FAMI_ESTRATOVIVIENDA_Estrato 3,FAMI_ESTRATOVIVIENDA_Estrato 4,FAMI_ESTRATOVIVIENDA_Estrato 5,FAMI_ESTRATOVIVIENDA_Estrato 6,FAMI_ESTRATOVIVIENDA_Sin Estrato,FAMI_EDUCACIONPADRE_Educación profesional incompleta,FAMI_EDUCACIONPADRE_Ninguno,FAMI_EDUCACIONPADRE_No Aplica,FAMI_EDUCACIONPADRE_No sabe,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Técnica o tecnológica completa,FAMI_EDUCACIONPADRE_Técnica o tecnológica incompleta,FAMI_EDUCACIONMADRE_Educación profesional incompleta,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta
692500,550236,20183,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
692501,98545,20203,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
692502,499179,20212,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
692503,782980,20195,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
692504,785185,20212,1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [26]:
train_listos.describe()

,ID,PERIODO,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,ESTU_PRGM_DEPARTAMENTO_ARAUCA,ESTU_PRGM_DEPARTAMENTO_ATLANTICO,ESTU_PRGM_DEPARTAMENTO_BOGOTÁ,ESTU_PRGM_DEPARTAMENTO_BOLIVAR,ESTU_PRGM_DEPARTAMENTO_BOYACA,ESTU_PRGM_DEPARTAMENTO_CALDAS,ESTU_PRGM_DEPARTAMENTO_CAQUETA,ESTU_PRGM_DEPARTAMENTO_CASANARE,ESTU_PRGM_DEPARTAMENTO_CAUCA,ESTU_PRGM_DEPARTAMENTO_CESAR,ESTU_PRGM_DEPARTAMENTO_CHOCO,ESTU_PRGM_DEPARTAMENTO_CORDOBA,ESTU_PRGM_DEPARTAMENTO_CUNDINAMARCA,ESTU_PRGM_DEPARTAMENTO_GUAVIARE,ESTU_PRGM_DEPARTAMENTO_HUILA,ESTU_PRGM_DEPARTAMENTO_LA GUAJIRA,ESTU_PRGM_DEPARTAMENTO_MAGDALENA,ESTU_PRGM_DEPARTAMENTO_META,ESTU_PRGM_DEPARTAMENTO_NARIÑO,ESTU_PRGM_DEPARTAMENTO_NORTE SANTANDER,ESTU_PRGM_DEPARTAMENTO_PUTUMAYO,ESTU_PRGM_DEPARTAMENTO_QUINDIO,ESTU_PRGM_DEPARTAMENTO_RISARALDA,ESTU_PRGM_DEPARTAMENTO_SAN ANDRES,ESTU_PRGM_DEPARTAMENTO_SANTANDER,ESTU_PRGM_DEPARTAMENTO_SUCRE,ESTU_PRGM_DEPARTAMENTO_TOLIMA,ESTU_PRGM_DEPARTAMENTO_VALLE,ESTU_PRGM_DEPARTAMENTO_VAUPES,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 2.5 millones y menos de 4 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 4 millones y menos de 5.5 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 5.5 millones y menos de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 500 mil y menos de 1 millón,ESTU_VALORMATRICULAUNIVERSIDAD_Menos de 500 mil,ESTU_VALORMATRICULAUNIVERSIDAD_Más de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_No pagó matrícula,FAMI_ESTRATOVIVIENDA_Estrato 2,FAMI_ESTRATOVIVIENDA_Estrato 3,FAMI_ESTRATOVIVIENDA_Estrato 4,FAMI_ESTRATOVIVIENDA_Estrato 5,FAMI_ESTRATOVIVIENDA_Estrato 6,FAMI_ESTRATOVIVIENDA_Sin Estrato,FAMI_EDUCACIONPADRE_Educación profesional incompleta,FAMI_EDUCACIONPADRE_Ninguno,FAMI_EDUCACIONPADRE_No Aplica,FAMI_EDUCACIONPADRE_No sabe,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Técnica o tecnológica completa,FAMI_EDUCACIONPADRE_Técnica o tecnológica incompleta,FAMI_EDUCACIONMADRE_Educación profesional incompleta,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta
count,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.00000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000
mean,494606.130576,20198.366679,0.894069,0.438702,0.120732,0.000851,0.059235,0.40745,0.029789,0.020286,0.016809,0.003840,0.002674,0.016565,0.011955,0.006194,0.017600,0.020243,0.000053,0.014433,0.006900,0.013736,0.011422,0.019428,0.032618,0.001148,0.011883,0.018309,0.000014,0.041629,0.012526,0.017214,0.064387,0.000020,0.184014,0.100702,0.055581,0.113652,0.115903,0.098215,0.028199,0.382394,0.304238,0.094605,0.034091,0.018202,0.004749,0.039110,0.031781,0.013327,0.023960,0.063782,0.080806,0.

In [27]:
test_listos.describe()

,ID,PERIODO,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,ESTU_PRGM_DEPARTAMENTO_ARAUCA,ESTU_PRGM_DEPARTAMENTO_ATLANTICO,ESTU_PRGM_DEPARTAMENTO_BOGOTÁ,ESTU_PRGM_DEPARTAMENTO_BOLIVAR,ESTU_PRGM_DEPARTAMENTO_BOYACA,ESTU_PRGM_DEPARTAMENTO_CALDAS,ESTU_PRGM_DEPARTAMENTO_CAQUETA,ESTU_PRGM_DEPARTAMENTO_CASANARE,ESTU_PRGM_DEPARTAMENTO_CAUCA,ESTU_PRGM_DEPARTAMENTO_CESAR,ESTU_PRGM_DEPARTAMENTO_CHOCO,ESTU_PRGM_DEPARTAMENTO_CORDOBA,ESTU_PRGM_DEPARTAMENTO_CUNDINAMARCA,ESTU_PRGM_DEPARTAMENTO_GUAVIARE,ESTU_PRGM_DEPARTAMENTO_HUILA,ESTU_PRGM_DEPARTAMENTO_LA GUAJIRA,ESTU_PRGM_DEPARTAMENTO_MAGDALENA,ESTU_PRGM_DEPARTAMENTO_META,ESTU_PRGM_DEPARTAMENTO_NARIÑO,ESTU_PRGM_DEPARTAMENTO_NORTE SANTANDER,ESTU_PRGM_DEPARTAMENTO_PUTUMAYO,ESTU_PRGM_DEPARTAMENTO_QUINDIO,ESTU_PRGM_DEPARTAMENTO_RISARALDA,ESTU_PRGM_DEPARTAMENTO_SAN ANDRES,ESTU_PRGM_DEPARTAMENTO_SANTANDER,ESTU_PRGM_DEPARTAMENTO_SUCRE,ESTU_PRGM_DEPARTAMENTO_TOLIMA,ESTU_PRGM_DEPARTAMENTO_VALLE,ESTU_PRGM_DEPARTAMENTO_VAUPES,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 2.5 millones y menos de 4 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 4 millones y menos de 5.5 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 5.5 millones y menos de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 500 mil y menos de 1 millón,ESTU_VALORMATRICULAUNIVERSIDAD_Menos de 500 mil,ESTU_VALORMATRICULAUNIVERSIDAD_Más de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_No pagó matrícula,FAMI_ESTRATOVIVIENDA_Estrato 2,FAMI_ESTRATOVIVIENDA_Estrato 3,FAMI_ESTRATOVIVIENDA_Estrato 4,FAMI_ESTRATOVIVIENDA_Estrato 5,FAMI_ESTRATOVIVIENDA_Estrato 6,FAMI_ESTRATOVIVIENDA_Sin Estrato,FAMI_EDUCACIONPADRE_Educación profesional incompleta,FAMI_EDUCACIONPADRE_Ninguno,FAMI_EDUCACIONPADRE_No Aplica,FAMI_EDUCACIONPADRE_No sabe,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Técnica o tecnológica completa,FAMI_EDUCACIONPADRE_Técnica o tecnológica incompleta,FAMI_EDUCACIONMADRE_Educación profesional incompleta,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta
count,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000
mean,494730.695238,20198.379078,0.894968,0.438397,0.120963,0.000832,0.059625,0.406738,0.029580,0.020274,0.017366,0.003784,0.002719,0.016342,0.012100,0.006220,0.017497,0.020274,0.000024,0.014718,0.007072,0.013976,0.011577,0.019229,0.032367,0.001230,0.011732,0.018033,0.000013,0.041100,0.012528,0.017339,0.064636,0.000024,0.185969,0.100594,0.055791,0.113092,0.116545,0.097919,0.028034,0.381025,0.305671,0.093748,0.034348,0.018367,0.004798,0.039571,0.032229,0.013326,0.023808,0.064986,0.081308,

In [50]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

In [51]:
label_encoder = LabelEncoder()
train_listos['RENDIMIENTO_GLOBAL'] = label_encoder.fit_transform(train_listos['RENDIMIENTO_GLOBAL'])

In [52]:
X = train_listos.drop(['RENDIMIENTO_GLOBAL', 'ID'], axis=1)
y = train_listos['RENDIMIENTO_GLOBAL']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [53]:
train_listos.head()

,ID,PERIODO,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,ESTU_PRGM_DEPARTAMENTO_ARAUCA,ESTU_PRGM_DEPARTAMENTO_ATLANTICO,ESTU_PRGM_DEPARTAMENTO_BOGOTÁ,ESTU_PRGM_DEPARTAMENTO_BOLIVAR,ESTU_PRGM_DEPARTAMENTO_BOYACA,ESTU_PRGM_DEPARTAMENTO_CALDAS,ESTU_PRGM_DEPARTAMENTO_CAQUETA,ESTU_PRGM_DEPARTAMENTO_CASANARE,ESTU_PRGM_DEPARTAMENTO_CAUCA,ESTU_PRGM_DEPARTAMENTO_CESAR,ESTU_PRGM_DEPARTAMENTO_CHOCO,ESTU_PRGM_DEPARTAMENTO_CORDOBA,ESTU_PRGM_DEPARTAMENTO_CUNDINAMARCA,ESTU_PRGM_DEPARTAMENTO_GUAVIARE,ESTU_PRGM_DEPARTAMENTO_HUILA,ESTU_PRGM_DEPARTAMENTO_LA GUAJIRA,ESTU_PRGM_DEPARTAMENTO_MAGDALENA,ESTU_PRGM_DEPARTAMENTO_META,ESTU_PRGM_DEPARTAMENTO_NARIÑO,ESTU_PRGM_DEPARTAMENTO_NORTE SANTANDER,ESTU_PRGM_DEPARTAMENTO_PUTUMAYO,ESTU_PRGM_DEPARTAMENTO_QUINDIO,ESTU_PRGM_DEPARTAMENTO_RISARALDA,ESTU_PRGM_DEPARTAMENTO_SAN ANDRES,ESTU_PRGM_DEPARTAMENTO_SANTANDER,ESTU_PRGM_DEPARTAMENTO_SUCRE,ESTU_PRGM_DEPARTAMENTO_TOLIMA,ESTU_PRGM_DEPARTAMENTO_VALLE,ESTU_PRGM_DEPARTAMENTO_VAUPES,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 2.5 millones y menos de 4 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 4 millones y menos de 5.5 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 5.5 millones y menos de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 500 mil y menos de 1 millón,ESTU_VALORMATRICULAUNIVERSIDAD_Menos de 500 mil,ESTU_VALORMATRICULAUNIVERSIDAD_Más de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_No pagó matrícula,FAMI_ESTRATOVIVIENDA_Estrato 2,FAMI_ESTRATOVIVIENDA_Estrato 3,FAMI_ESTRATOVIVIENDA_Estrato 4,FAMI_ESTRATOVIVIENDA_Estrato 5,FAMI_ESTRATOVIVIENDA_Estrato 6,FAMI_ESTRATOVIVIENDA_Sin Estrato,FAMI_EDUCACIONPADRE_Educación profesional incompleta,FAMI_EDUCACIONPADRE_Ninguno,FAMI_EDUCACIONPADRE_No Aplica,FAMI_EDUCACIONPADRE_No sabe,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Técnica o tecnológica completa,FAMI_EDUCACIONPADRE_Técnica o tecnológica incompleta,FAMI_EDUCACIONMADRE_Educación profesional incompleta,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta,RENDIMIENTO_GLOBAL
0,904256,20212,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1,645256,20212,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
2,308367,20203,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
3,470353,20195,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
4,989032,20212,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3


In [54]:
train_listos.shape

(692500, 70)

In [55]:
from tqdm import tqdm
from time import sleep  # Solo para simular tiempos de ejecución
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# Modelo ajustado con hiperparámetros optimizados
rf_model = RandomForestClassifier(
    n_estimators=150,           # Mayor estabilidad
    criterion='gini',           # Alternativa: "entropy" o "log_loss"
    max_depth=None,             # Sin límite inicial
    min_samples_split=5,        # Reduce el sobreajuste
    min_samples_leaf=2,         # Hoja mínima más pequeña
    max_features='sqrt',        # Acelera el entrenamiento
    bootstrap=True,             # Subconjuntos aleatorios
    class_weight='balanced',    # Balancea las clases
    random_state=42             # Reproducibilidad
)

# Entrenamiento
rf_model.fit(X_train, y_train)


RandomForestClassifier(class_weight='balanced', min_samples_leaf=2,
                       min_samples_split=5, n_estimators=150, random_state=42)

In [56]:
y_pred_rf = rf_model.predict(X_test)

In [57]:
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Random Forest Reporte de Clasificacion:\n", classification_report(y_test, y_pred_rf))

Random Forest Accuracy: 0.3916028880866426
Random Forest Reporte de Clasificacion:
               precision    recall  f1-score   support

           0       0.51      0.57      0.53     35165
           1       0.40      0.50      0.45     34573
           2       0.30      0.23      0.26     34259
           3       0.30      0.26      0.28     34503

    accuracy                           0.39    138500
   macro avg       0.38      0.39      0.38    138500
weighted avg       0.38      0.39      0.38    138500



In [58]:
ids = test_listos['ID']

new_data_processed = test_listos.drop('ID', axis=1)

In [59]:
new_data_processed.head()

,PERIODO,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,ESTU_PRGM_DEPARTAMENTO_ARAUCA,ESTU_PRGM_DEPARTAMENTO_ATLANTICO,ESTU_PRGM_DEPARTAMENTO_BOGOTÁ,ESTU_PRGM_DEPARTAMENTO_BOLIVAR,ESTU_PRGM_DEPARTAMENTO_BOYACA,ESTU_PRGM_DEPARTAMENTO_CALDAS,ESTU_PRGM_DEPARTAMENTO_CAQUETA,ESTU_PRGM_DEPARTAMENTO_CASANARE,ESTU_PRGM_DEPARTAMENTO_CAUCA,ESTU_PRGM_DEPARTAMENTO_CESAR,ESTU_PRGM_DEPARTAMENTO_CHOCO,ESTU_PRGM_DEPARTAMENTO_CORDOBA,ESTU_PRGM_DEPARTAMENTO_CUNDINAMARCA,ESTU_PRGM_DEPARTAMENTO_GUAVIARE,ESTU_PRGM_DEPARTAMENTO_HUILA,ESTU_PRGM_DEPARTAMENTO_LA GUAJIRA,ESTU_PRGM_DEPARTAMENTO_MAGDALENA,ESTU_PRGM_DEPARTAMENTO_META,ESTU_PRGM_DEPARTAMENTO_NARIÑO,ESTU_PRGM_DEPARTAMENTO_NORTE SANTANDER,ESTU_PRGM_DEPARTAMENTO_PUTUMAYO,ESTU_PRGM_DEPARTAMENTO_QUINDIO,ESTU_PRGM_DEPARTAMENTO_RISARALDA,ESTU_PRGM_DEPARTAMENTO_SAN ANDRES,ESTU_PRGM_DEPARTAMENTO_SANTANDER,ESTU_PRGM_DEPARTAMENTO_SUCRE,ESTU_PRGM_DEPARTAMENTO_TOLIMA,ESTU_PRGM_DEPARTAMENTO_VALLE,ESTU_PRGM_DEPARTAMENTO_VAUPES,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 2.5 millones y menos de 4 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 4 millones y menos de 5.5 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 5.5 millones y menos de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 500 mil y menos de 1 millón,ESTU_VALORMATRICULAUNIVERSIDAD_Menos de 500 mil,ESTU_VALORMATRICULAUNIVERSIDAD_Más de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_No pagó matrícula,FAMI_ESTRATOVIVIENDA_Estrato 2,FAMI_ESTRATOVIVIENDA_Estrato 3,FAMI_ESTRATOVIVIENDA_Estrato 4,FAMI_ESTRATOVIVIENDA_Estrato 5,FAMI_ESTRATOVIVIENDA_Estrato 6,FAMI_ESTRATOVIVIENDA_Sin Estrato,FAMI_EDUCACIONPADRE_Educación profesional incompleta,FAMI_EDUCACIONPADRE_Ninguno,FAMI_EDUCACIONPADRE_No Aplica,FAMI_EDUCACIONPADRE_No sabe,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Técnica o tecnológica completa,FAMI_EDUCACIONPADRE_Técnica o tecnológica incompleta,FAMI_EDUCACIONMADRE_Educación profesional incompleta,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta
692500,20183,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
692501,20203,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
692502,20212,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
692503,20195,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
692504,20212,1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [60]:

predictions = rf_model.predict(new_data_processed)

test_listos['RENDIMIENTO_GLOBAL'] = predictions


<ipython-input-60-443714cec19f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_listos['RENDIMIENTO_GLOBAL'] = predictions


In [61]:
test_listos.head()

,ID,PERIODO,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,ESTU_PRGM_DEPARTAMENTO_ARAUCA,ESTU_PRGM_DEPARTAMENTO_ATLANTICO,ESTU_PRGM_DEPARTAMENTO_BOGOTÁ,ESTU_PRGM_DEPARTAMENTO_BOLIVAR,ESTU_PRGM_DEPARTAMENTO_BOYACA,ESTU_PRGM_DEPARTAMENTO_CALDAS,ESTU_PRGM_DEPARTAMENTO_CAQUETA,ESTU_PRGM_DEPARTAMENTO_CASANARE,ESTU_PRGM_DEPARTAMENTO_CAUCA,ESTU_PRGM_DEPARTAMENTO_CESAR,ESTU_PRGM_DEPARTAMENTO_CHOCO,ESTU_PRGM_DEPARTAMENTO_CORDOBA,ESTU_PRGM_DEPARTAMENTO_CUNDINAMARCA,ESTU_PRGM_DEPARTAMENTO_GUAVIARE,ESTU_PRGM_DEPARTAMENTO_HUILA,ESTU_PRGM_DEPARTAMENTO_LA GUAJIRA,ESTU_PRGM_DEPARTAMENTO_MAGDALENA,ESTU_PRGM_DEPARTAMENTO_META,ESTU_PRGM_DEPARTAMENTO_NARIÑO,ESTU_PRGM_DEPARTAMENTO_NORTE SANTANDER,ESTU_PRGM_DEPARTAMENTO_PUTUMAYO,ESTU_PRGM_DEPARTAMENTO_QUINDIO,ESTU_PRGM_DEPARTAMENTO_RISARALDA,ESTU_PRGM_DEPARTAMENTO_SAN ANDRES,ESTU_PRGM_DEPARTAMENTO_SANTANDER,ESTU_PRGM_DEPARTAMENTO_SUCRE,ESTU_PRGM_DEPARTAMENTO_TOLIMA,ESTU_PRGM_DEPARTAMENTO_VALLE,ESTU_PRGM_DEPARTAMENTO_VAUPES,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 2.5 millones y menos de 4 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 4 millones y menos de 5.5 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 5.5 millones y menos de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 500 mil y menos de 1 millón,ESTU_VALORMATRICULAUNIVERSIDAD_Menos de 500 mil,ESTU_VALORMATRICULAUNIVERSIDAD_Más de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_No pagó matrícula,FAMI_ESTRATOVIVIENDA_Estrato 2,FAMI_ESTRATOVIVIENDA_Estrato 3,FAMI_ESTRATOVIVIENDA_Estrato 4,FAMI_ESTRATOVIVIENDA_Estrato 5,FAMI_ESTRATOVIVIENDA_Estrato 6,FAMI_ESTRATOVIVIENDA_Sin Estrato,FAMI_EDUCACIONPADRE_Educación profesional incompleta,FAMI_EDUCACIONPADRE_Ninguno,FAMI_EDUCACIONPADRE_No Aplica,FAMI_EDUCACIONPADRE_No sabe,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Técnica o tecnológica completa,FAMI_EDUCACIONPADRE_Técnica o tecnológica incompleta,FAMI_EDUCACIONMADRE_Educación profesional incompleta,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta,RENDIMIENTO_GLOBAL
692500,550236,20183,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
692501,98545,20203,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2
692502,499179,20212,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3
692503,782980,20195,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
692504,785185,20212,1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1


In [70]:
test = test_listos[['ID', 'RENDIMIENTO_GLOBAL']]
test.head()

,ID,RENDIMIENTO_GLOBAL
692500,550236,0
692501,98545,2
692502,499179,3
692503,782980,1
692504,785185,1


In [71]:
mapping = {2: 'medio-alto', 0: 'alto', 1: 'bajo', 3: 'medio-bajo'}

# Aplicar el mapeo a la columna RENDIMIENTO_GLOBAL
test['RENDIMIENTO_GLOBAL'] = test['RENDIMIENTO_GLOBAL'].map(mapping)

<ipython-input-71-3fb4d68f0492>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['RENDIMIENTO_GLOBAL'] = test['RENDIMIENTO_GLOBAL'].map(mapping)


In [73]:
test.to_csv('test_primermodelo.csv', index=False)

In [74]:
test.shape

(296786, 2)